In [ ]:
%%html
<style>
.container {
  width: 100%;
}
</style>

In [ ]:
%load_ext nb_black
%load_ext nb_mypy

In [ ]:
import nbimporter
from Exercise06AI import Exercise06AI

# Aufgabe 07Lite: Minimax mit Alpha-Beta-Pruning und Progressive Deepening OHNE Memoisierung

Dieses Notebook ist eine Kopie der `Exercise07AI` ohne Memoisierung. Durch diese Modifizierung sinkt der verwendete Arbeitsspeicher erheblich und die Berechnungsdauer erhöht sich leicht.

In [ ]:
import chess
from typing import Any


class Exercise07LiteAI(Exercise06AI):
    """Chooses middle game moves using minimax algorithm, alpha-beta-pruning and memoization"""

    def __init__(self, **kwargs) -> None:
        super().__init__(**kwargs)
        self.local_cache: dict[tuple, int] = {}

## Entfernung der Memoisierung

Diese Klasse ist bis auf das Entfernen der Memoisierung identisch zur Klasse `Exercise07AI`.

In [ ]:
from typing import Any
import heapq


class Exercise07LiteAI(Exercise07LiteAI):  # type: ignore
    def minimax(
        self,
        board: chess.Board,
        depth: int,
        current_evaluation: int,
        alpha: int = -Exercise07LiteAI.LIMIT,
        beta: int = Exercise07LiteAI.LIMIT,
    ) -> tuple[int, chess.Move]:
        """Searches the best value with given depth using minimax algorithm"""
        early_abort_evaluation = self.minimax_early_abort(
            board, depth, current_evaluation
        )
        if early_abort_evaluation is not None:
            return early_abort_evaluation, None

        best_move = None

        # White to play (positive numbers are good)
        moves: list[tuple[int, int, chess.Move]] = []
        if board.turn:
            for i, move in enumerate(board.legal_moves):
                old_eval = self.local_cache.get((depth - 2, move), i)
                heapq.heappush(moves, (-old_eval, i, move))
            maxEvaluation = alpha
            while moves:
                move = heapq.heappop(moves)[2]
                board.push(move)
                evaluation, _ = self.minimax(
                    board,
                    depth - 1,
                    self.evaluate(board, current_evaluation),
                    maxEvaluation,
                    beta,
                )
                self.local_cache[(depth - 1, move)] = evaluation
                board.pop()
                if evaluation >= beta:
                    return evaluation, move
                if depth == self.DEPTH and evaluation > maxEvaluation:
                    best_move = move
                maxEvaluation = max(maxEvaluation, evaluation)
            return maxEvaluation, best_move

        # Black to play (negative numbers are good)
        else:
            for i, move in enumerate(board.legal_moves):
                old_eval = self.local_cache.get((depth - 2, move), i)
                heapq.heappush(moves, (old_eval, i, move))
            minEvaluation = beta
            while moves:
                move = heapq.heappop(moves)[2]
                board.push(move)
                evaluation, _ = self.minimax(
                    board,
                    depth - 1,
                    self.evaluate(board, current_evaluation),
                    alpha,
                    minEvaluation,
                )
                self.local_cache[(depth - 1, move)] = evaluation
                board.pop()
                if evaluation <= alpha:
                    return evaluation, move
                if depth == self.DEPTH and evaluation < minEvaluation:
                    best_move = move
                minEvaluation = min(minEvaluation, evaluation)
            return minEvaluation, best_move

In [ ]:
class Exercise07LiteAI(Exercise07LiteAI):  # type: ignore
    def get_next_middle_game_move(self, board: chess.Board) -> chess.Move:
        """Gets the best next move"""
        self.last_evaluation: int | None  # type annotation for mypy
        if self.is_king_endgame != self.check_king_endgame(board):
            self.last_evaluation += self.get_endgame_evaluation_change(board)
        # Calculate current evaluation
        if self.last_evaluation is None:  # type: ignore
            current_evaluation = self.full_evaluate(board)
        else:
            # Get current evaluation (after opponent move)
            current_evaluation = self.evaluate(board, self.last_evaluation)

        for depth in range(1, self.DEPTH + 1):
            # Call minimax and get best move
            _, best_move = self.minimax(board, depth, current_evaluation)

        # Reset local cache
        self.local_cache: dict[tuple, int] = {}

        # Debugging fail save
        assert best_move, f"""
        Best move is None with fen '{board.fen()}' at player {type(self).__name__}! 
        depth: {self.DEPTH}, last_eval: {self.last_evaluation}, current_evaluation: {current_evaluation},
        is_king_engame: {getattr(self, 'is_king_endgame', "N/A")}, move_stack: {board.move_stack}
        """
        # Update last evaluation (after player move)
        self.last_evaluation = current_evaluation + self.incremental_evaluate(
            board, best_move
        )
        return best_move

## Debugging Bereich

Die folgenden Zellen enthalten Code zum Testen der oben implementierten Funktionen.

In [ ]:
import Exercise03AI

In [ ]:
# Create player and board
unit_test_player = Exercise07LiteAI(player_name="Ex07LiteAI", search_depth=2)
board = chess.Board("5rk1/1b3p2/8/3p4/3p2P1/2Q4B/5P1K/R3R3 b - - 0 36")
board

In [ ]:
# Test minimax
Exercise03AI.test_minimax(unit_test_player, board)

In [ ]:
# Test next move function
Exercise03AI.test_next_move(unit_test_player, board)

## Temporärer Bereich

Der folgende Bereich dient zum temporären Debuggen und kann nicht-funktionierenden Code enthalten. Dieser Bereich wird vor der Abgabe entfernt.